In [12]:
import os
import re
import sys
sys.path.append('../')
import json
import pywt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import simpson
from sklearn.linear_model import RidgeClassifier
from tqdm import tqdm

from transform import TimeSeriesTransform

In [13]:
cfg = json.load(open("../config/config.json"))

ts_trans = TimeSeriesTransform(cfg)

In [14]:
INSTANCES_DIR = '../data/linear_acuator/instances/'
INFERENCE_DIR = '../data/linear_acuator/inference/'
STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']

In [15]:
load = '20kg'
filenames = [os.listdir(os.path.join(INSTANCES_DIR, load, state)) for state in STATES]
filenames = [filename for sublist in filenames for filename in sublist]

In [16]:
X, y = [], []
for filename in filenames:
    load_num = load[:2]
    state = re.match(fr'(.*)_{load_num}', filename).group(1)
    df = pd.read_csv(os.path.join(INSTANCES_DIR, load, state, filename))
    tmp_cur = ts_trans.smoothing(ts_df=df, field='current')
    # tmp_pos = ts_transform.smoothing(ts_df=df, field='position_error')
    X.append(tmp_cur)
    y.append(state)

In [17]:
# Define scales and perform Continuous Wavelet Transform (CWT)
wavelet = 'morl'  # Morlet wavelet for CWT
scales = np.arange(1, 1001)  # Range of scales to use for the CWT
raven_results = []
for i in tqdm(range(len(X))):
    coefficients, frequencies = pywt.cwt(X[i], scales, wavelet, sampling_period=1/400)
    integral_simps = simpson(coefficients)
    raven_results.append(integral_simps)

raven_results = np.array(raven_results)
print(raven_results.shape)

100%|██████████| 455/455 [04:17<00:00,  1.77it/s]

(455, 1000)


In [18]:
ridge = RidgeClassifier()
ridge.fit(raven_results, y)
ridge.score(raven_results, y)

0.8351648351648352